In [1]:
from datasets import load_dataset
import re
import random

In [2]:
patterns = [
        r"Imagine you are (?:an?|the)? ([^.,\n]+)",  # Matches "Imagine you are a/an/the X"
        r"As (?:an?|the)? ([^.,\n]+)",              # Matches "As a/an/the X"
        r"You are (?:an?|the)? ([^.,\n]+)",         # Matches "You are a/an/the X"
        r"Act as (?:an?|the)? ([^.,\n]+)",          # Matches "Act as a/an/the X"
        r"the perspective of (?:an?|the)? ([^.,\n]+)",  # Matches "Take on the role of a/an/the X"
        r"Take on the role of (?:an?|the)? ([^.,\n]+)",  # Matches "Take on the role of a/an/the X"
        r"Respond as (?:an?|the)? ([^.,\n]+)",      # Matches "Respond as a/an/the X"
        r"Answer as (?:an?|the)? ([^.,\n]+)",       # Matches "Answer as a/an/the X"
    ]

match = re.match(patterns[1], "As a mother who is, not great")
match.group(1)

'mother who is'

In [3]:
dataset = load_dataset("allenai/tulu-3-sft-personas-instruction-following")["train"]
dataset

Dataset({
    features: ['id', 'prompt', 'messages', 'constraints'],
    num_rows: 29980
})

In [4]:
def find_persona(row):
    persona = None
    for patt in patterns:
        match = re.match(patt, row["prompt"], re.IGNORECASE)
        if match is not None:
            if match.group(1) is not None:
                persona = " ".join(match.group(1).strip().split())
                break
    row['persona'] = persona
    
    return row

In [5]:
dataset = dataset.map(find_persona)
persona_data=dataset.filter(lambda row: row['persona'] is not None)
persona_data

Dataset({
    features: ['id', 'prompt', 'messages', 'constraints', 'persona'],
    num_rows: 2947
})

In [ ]:
dataset.select(random.sample(range(len(dataset)), k=5))['prompt']

In [ ]:
list(zip(persona_data['persona'], persona_data['prompt']))

In [ ]:
persona_data.filter(lambda row: row['persona']=="older South Sudanese immigrant who")['prompt']

These personas are derived from Persona Hub, which is all synthetic, and we end up with examples like the above, which are definitely weird to finetune as a persona? These are way too fine-grained!

## Dolly 15k?

In [ ]:
dataset = load_dataset("databricks/databricks-dolly-15k")["train"]
dataset

In [ ]:
def find_persona2(row):
    persona = None
    for patt in patterns:
        match = re.match(patt, row["instruction"], re.IGNORECASE)
        if match is not None:
            if match.group(1) is not None:
                persona = " ".join(match.group(1).strip().split())
                break
    row['persona'] = str(persona)
    
    return row
    
dataset = dataset.map(find_persona2)

In [ ]:
persona_data=dataset.filter(lambda row: row['persona'] != "None")
persona_data

In [ ]:
persona_data['persona']

In [ ]:
dataset.select(random.sample(range(len(dataset)), k=5))['instruction']